### Import Dependencies

In [ ]:
from pydantic import BaseModel, Field

from qdrant_client import QdrantClient
from qdrant_client.models import Prefetch, Filter, FieldCondition, MatchText, FusionQuery, Document


from langsmith import traceable, get_current_run_tree

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.types import Send, Command

from langchain_core.messages import AIMessage, ToolMessage, convert_to_openai_messages

from jinja2 import Template
from typing import Literal, Dict, Any, Annotated, List, Optional, Sequence
from IPython.display import Image, display
from operator import add
from openai import OpenAI

import openai

import random
import ast
import inspect
import instructor
import json

from utils.utils import get_tool_descriptions, format_ai_message
from utils.tools import add_to_shopping_cart, get_shopping_cart, remove_from_cart, get_formatted_items_context, get_formatted_reviews_context

from langgraph.checkpoint.postgres import PostgresSaver
import psycopg2
from psycopg2.extras import RealDictCursor, execute_batch
import numpy as np

### Fictional Warehouses

In [ ]:
warehouses = [
    {
        "warehouse_id": "DE-BER-01",
        "warehouse_location": "Berlin, Germany",
        "warehouse_name": "Berlin Distribution Center"
    },
    {
        "warehouse_id": "DE-MUN-01",
        "warehouse_location": "Munich, Germany",
        "warehouse_name": "Munich Logistics Hub"
    },
    {
        "warehouse_id": "DE-HAM-01",
        "warehouse_location": "Hamburg, Germany",
        "warehouse_name": "Hamburg North Warehouse"
    },
    {
        "warehouse_id": "FR-PAR-01",
        "warehouse_location": "Paris, France",
        "warehouse_name": "Paris Central Depot"
    },
    {
        "warehouse_id": "FR-LYO-01",
        "warehouse_location": "Lyon, France",
        "warehouse_name": "Lyon Regional Warehouse"
    },
    {
        "warehouse_id": "FR-MAR-01",
        "warehouse_location": "Marseille, France",
        "warehouse_name": "Marseille Mediterranean Hub"
    }
]

### Simulate Stock Availability for each of the warehouse

#### Retrieve all item IDs from Amazon items Qdrant Collection

In [ ]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [ ]:
dummy_vector = np.zeros(1536).tolist()

In [ ]:
payload = qdrant_client.query_points(
    collection_name="Amazon-items-collection-01-hybrid-search",
    query=dummy_vector,
    using="text-embedding-3-small",
    limit=1000,
    with_payload=["parent_asin"],
    with_vectors=False
)

In [ ]:
payload.points

In [ ]:
parent_asin_list = [point.payload["parent_asin"] for point in payload.points]

In [ ]:
parent_asin_list

In [ ]:
len(parent_asin_list)

### Generate Synthetic availability for all items in Qdrant

In [ ]:
def generate_inventory_data(warehouses, product_ids, availability_rate=0.75):
    
    inventory_records = []
    
    for warehouse in warehouses:
        for product_id in product_ids:
            # 75% chance the product is available in this warehouse
            if random.random() < availability_rate:
                total_quantity = random.randint(0, 100)
                
                # Only add to inventory if quantity > 0
                if total_quantity > 0:
                    inventory_records.append({
                        "warehouse_id": warehouse["warehouse_id"],
                        "warehouse_location": warehouse["warehouse_location"],
                        "warehouse_name": warehouse["warehouse_name"],
                        "product_id": product_id,
                        "total_quantity": total_quantity,
                        "reserved_quantity": 0  # Starting with no reservations
                    })
    
    return inventory_records

In [ ]:
inventory_data = generate_inventory_data(warehouses, parent_asin_list, availability_rate=0.75)

In [ ]:
inventory_data

In [ ]:
1000*0.75*6

In [ ]:
len(inventory_data)

### Write syntehetic data to Postgres

In [ ]:
def insert_inventory_to_db(inventory_records):
   
    try:
        # Connect to the database
        conn = psycopg2.connect(
            host="localhost",
            port=5433,
            database="tools_database",
            user="langgraph_user",
            password="langgraph_password"
        )
        conn.autocommit = True

        with conn.cursor(cursor_factory=RealDictCursor) as cursor:
        
            # Prepare the INSERT query
            insert_query = """
            INSERT INTO warehouses.inventory 
            (warehouse_id, warehouse_location, warehouse_name, product_id, total_quantity, reserved_quantity)
            VALUES (%(warehouse_id)s, %(warehouse_location)s, %(warehouse_name)s, %(product_id)s, %(total_quantity)s, %(reserved_quantity)s)
            """
            
            # Use execute_batch for better performance with many inserts
            execute_batch(cursor, insert_query, inventory_records, page_size=100)
            
            # Commit the transaction
            conn.commit()
            
            print(f"Successfully inserted {len(inventory_records)} records into warehouses.inventory")
            
            # Close cursor and connection
            cursor.close()
            conn.close()
        
    except psycopg2.Error as e:
        print(f"Database error: {e}")
        if conn:
            conn.rollback()
    except Exception as e:
        print(f"Error: {e}")
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [ ]:
insert_inventory_to_db(inventory_data)